# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

## Import

In [20]:
import sqlite3
import pandas as pd

## Connections

In [21]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

## Table: Matches

- Match_ID (int): unique ID per match
- Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)
- Season (int): Season the match took place in (usually covering the period of August till May of the following year)
- Date (str): Date of the match
- HomeTeam (str): Name of the home team
- AwayTeam (str): Name of the away team
- FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
- FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
- FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

In [78]:
# Matches from the 2011 season.

cur.execute("""SELECT * FROM matches WHERE season = 2011;""")
df_matches = pd.DataFrame(cur.fetchall())
df_matches.columns = [i[0] for i in cur.description]
display(df_matches.head())
display(df_matches.info())
# display(df_matches.isna().sum())
# display(df_matches.describe())
df_matches.Date.sort_values()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 9 columns):
Match_ID    992 non-null int64
Div         992 non-null object
Season      992 non-null int64
Date        992 non-null object
HomeTeam    992 non-null object
AwayTeam    992 non-null object
FTHG        992 non-null int64
FTAG        992 non-null int64
FTR         992 non-null object
dtypes: int64(4), object(5)
memory usage: 69.8+ KB


None

41     2011-07-15
75     2011-07-15
459    2011-07-15
586    2011-07-16
458    2011-07-16
350    2011-07-17
54     2011-07-17
510    2011-07-17
268    2011-07-18
413    2011-07-22
504    2011-07-22
575    2011-07-22
433    2011-07-23
169    2011-07-23
607    2011-07-24
482    2011-07-24
298    2011-07-24
371    2011-07-25
592    2011-08-05
127    2011-08-05
521    2011-08-05
453    2011-08-05
145    2011-08-06
102    2011-08-06
38     2011-08-06
374    2011-08-06
200    2011-08-06
341    2011-08-06
219    2011-08-06
556    2011-08-06
          ...    
178    2012-05-05
115    2012-05-05
518    2012-05-06
979    2012-05-06
978    2012-05-06
977    2012-05-06
976    2012-05-06
975    2012-05-06
591    2012-05-06
973    2012-05-06
974    2012-05-06
66     2012-05-06
43     2012-05-06
264    2012-05-06
436    2012-05-06
448    2012-05-06
460    2012-05-06
76     2012-05-06
980    2012-05-07
981    2012-05-08
989    2012-05-13
988    2012-05-13
987    2012-05-13
986    2012-05-13
982    201

In [23]:
cur.execute("""SELECT * FROM sqlite_sequence;""")
df_sqlite_sequence = pd.DataFrame(cur.fetchall())
df_sqlite_sequence.columns = [i[0] for i in cur.description]
display(df_sqlite_sequence.head())
display(df_sqlite_sequence.info())
display(df_sqlite_sequence.isna().sum())

,name,seq
0,Matches,46774
1,Unique_Teams,128


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
name    2 non-null object
seq     2 non-null int64
dtypes: int64(1), object(1)
memory usage: 112.0+ bytes


None

name    0
seq     0
dtype: int64

## Table: Teams

- Season (str): Football season for which the data is valid
- TeamName (str): Name of the team the data concerns
- KaderHome (str): Number of Players in the squad
- AvgAgeHome (str): Average age of players
- ForeignPlayersHome (str): Number of foreign players (non-German, non-English respectively) playing for the team
- OverallMarketValueHome (str): Overall market value of the team pre-season in EUR (based on data from transfermarkt.de)
- AvgMarketValueHome (str): Average market value (per player) of the team pre-season in EUR (based on data from transfermarkt.de)
- StadiumCapacity (str): Maximum stadium capacity of the team's home stadium

In [24]:
# Teams in 2011 season.

cur.execute("""SELECT * FROM teams WHERE season = 2011;""")
df_teams = pd.DataFrame(cur.fetchall())
df_teams.columns = [i[0] for i in cur.description]
display(df_teams.head())
display(df_teams.info())
display(df_teams.isna().sum())
display(df_teams.describe())

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2011,Bayern Munich,24,25,11,335600000,13980000,75000
1,2011,Dortmund,28,24,12,158200000,5650000,81359
2,2011,Leverkusen,29,23,9,151100000,5210000,30210
3,2011,Schalke 04,37,24,20,136730000,3700000,62271
4,2011,Werder Bremen,38,23,17,125350000,3300000,42100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 8 columns):
Season                    36 non-null object
TeamName                  36 non-null object
KaderHome                 36 non-null object
AvgAgeHome                36 non-null object
ForeignPlayersHome        36 non-null object
OverallMarketValueHome    36 non-null object
AvgMarketValueHome        36 non-null object
StadiumCapacity           36 non-null object
dtypes: object(8)
memory usage: 2.3+ KB


None

Season                    0
TeamName                  0
KaderHome                 0
AvgAgeHome                0
ForeignPlayersHome        0
OverallMarketValueHome    0
AvgMarketValueHome        0
StadiumCapacity           0
dtype: int64

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
count,36,36,36,36,36,36,36,36
unique,1,36,14,4,16,36,36,19
top,2011,Kaiserslautern,32,24,11,59080000,534000,47728
freq,36,1,5,19,4,1,1,18


## Table: Teams_in_Matches

- Match_ID (int): Unique match ID
- Unique_Team_ID (int): Unique team ID (This table is used to easily retrieve each match a given team has played in)

In [25]:
cur.execute("""SELECT * FROM teams_in_matches;""")
df_teams_in_matches = pd.DataFrame(cur.fetchall())
df_teams_in_matches.columns = [i[0] for i in cur.description]
display(df_teams.head())
display(df_teams.info())
display(df_teams_in_matches.isna().sum())
display(df_teams_in_matches.describe())

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2011,Bayern Munich,24,25,11,335600000,13980000,75000
1,2011,Dortmund,28,24,12,158200000,5650000,81359
2,2011,Leverkusen,29,23,9,151100000,5210000,30210
3,2011,Schalke 04,37,24,20,136730000,3700000,62271
4,2011,Werder Bremen,38,23,17,125350000,3300000,42100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 8 columns):
Season                    36 non-null object
TeamName                  36 non-null object
KaderHome                 36 non-null object
AvgAgeHome                36 non-null object
ForeignPlayersHome        36 non-null object
OverallMarketValueHome    36 non-null object
AvgMarketValueHome        36 non-null object
StadiumCapacity           36 non-null object
dtypes: object(8)
memory usage: 2.3+ KB


None

Match_ID          0
Unique_Team_ID    0
dtype: int64

,Match_ID,Unique_Team_ID
count,49148.000000,49148.000000
mean,27325.139924,52.903028
std,16647.432504,38.825678
min,1.000000,1.000000
25%,6144.000000,17.000000
50%,34485.500000,42.000000
75%,40631.000000,90.000000
max,46774.000000,128.000000


## Table: Unique Teams

- TeamName (str): Name of a team
- Unique_Team_ID (int): Unique identifier for each team

In [26]:
cur.execute("""SELECT * FROM unique_teams;""")
df_unique_teams = pd.DataFrame(cur.fetchall())
df_unique_teams.columns = [i[0] for i in cur.description]
display(df_unique_teams.head())
display(df_unique_teams.info())
display(df_unique_teams.isna().sum())

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
TeamName          128 non-null object
Unique_Team_ID    128 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.1+ KB


None

TeamName          0
Unique_Team_ID    0
dtype: int64

## The Goal

- The name of the team
- The total number of goals scored by the team during the 2011 season
- The total number of wins the team earned during the 2011 season
- A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
- The team's win percentage on days where it was raining during games in the 2011 season.

## Team Name and Total Number of Goals per Team in 2011

In [27]:
# DataFrame df_goals_home is grouped by df_matches HomeTeam and total goals when they were the home team.

df_goals_home = pd.DataFrame(df_matches.groupby('HomeTeam').FTHG.sum().reset_index())
df_goals_home.head()

,HomeTeam,FTHG
0,Aachen,15
1,Arsenal,39
2,Aston Villa,20
3,Augsburg,20
4,Bayern Munich,49


In [28]:
# DataFrame df_goals_away is grouped by df_matches AwayTeam and total goals when they were the away team.

df_goals_away = pd.DataFrame(df_matches.groupby('AwayTeam').FTAG.sum().reset_index())
df_goals_away.head()

,AwayTeam,FTAG
0,Aachen,15
1,Arsenal,35
2,Aston Villa,17
3,Augsburg,16
4,Bayern Munich,28


In [29]:
# Merged or joined DataFrames df_home_team and df_away_team as a new DataFrame df_goals.

df_goals = df_goals_home.merge(df_goals_away, how = 'outer', left_on = 'HomeTeam', right_on = 'AwayTeam')
df_goals.head()

,HomeTeam,FTHG,AwayTeam,FTAG
0,Aachen,15,Aachen,15
1,Arsenal,39,Arsenal,35
2,Aston Villa,20,Aston Villa,17
3,Augsburg,20,Augsburg,16
4,Bayern Munich,49,Bayern Munich,28


In [30]:
# Created a new column in df_goals titled Goals that adds the goals from FTHG and FTAG.

df_goals['Goals'] = df_goals.FTHG + df_goals.FTAG
df_goals.head()

,HomeTeam,FTHG,AwayTeam,FTAG,Goals
0,Aachen,15,Aachen,15,30
1,Arsenal,39,Arsenal,35,74
2,Aston Villa,20,Aston Villa,17,37
3,Augsburg,20,Augsburg,16,36
4,Bayern Munich,49,Bayern Munich,28,77


In [31]:
# Dropped three columns in df_goals FTHG, AwayTeam, and FTAG.

df_goals = df_goals.drop(columns = ['FTHG', 'AwayTeam', 'FTAG'])
df_goals.head()

,HomeTeam,Goals
0,Aachen,30
1,Arsenal,74
2,Aston Villa,37
3,Augsburg,36
4,Bayern Munich,77


In [32]:
# Renamed the column in df_goals HomeTeam to Teams.

df_goals = df_goals.rename(columns = {'HomeTeam': 'Teams'})
df_goals.head()

,Teams,Goals
0,Aachen,30
1,Arsenal,74
2,Aston Villa,37
3,Augsburg,36
4,Bayern Munich,77


## Number of Wins  in 2011

In [33]:
# DataFrame df_wins_home filters for home wins.

df_wins_home = df_matches[df_matches['FTR'] == 'H']
df_wins_home.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
5,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H
9,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H
11,1103,D1,2011,2012-03-03,Leverkusen,Bayern Munich,2,0,H
16,1108,D1,2011,2012-04-11,Dortmund,Bayern Munich,1,0,H


In [34]:
# df_matches.head()

In [35]:
# DataFrame df_wins_away filters for away wins.

df_wins_away= df_matches[df_matches['FTR'] == 'A']
df_wins_away.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
7,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A
8,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A


In [36]:
# DataFrame df_loses_away filters for away loss.

df_loses_away = df_matches[df_matches['FTR'] == 'H']
df_loses_away.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
5,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H
9,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H
11,1103,D1,2011,2012-03-03,Leverkusen,Bayern Munich,2,0,H
16,1108,D1,2011,2012-04-11,Dortmund,Bayern Munich,1,0,H


In [37]:
# Overwrote df_loses_away with a groupby of HomeTeam and count of each win.

df_loses_home = pd.DataFrame(df_loses_home.groupby('HomeTeam').FTR.count().reset_index())
df_loses_home.head()

NameError: name 'df_loses_home' is not defined

In [38]:
# Overwrote df_loses_home with a groupby of HomeTeam and count of each win.

df_wins_home = pd.DataFrame(df_wins_home.groupby('HomeTeam').FTR.count().reset_index())
df_wins_home.head()

,HomeTeam,FTR
0,Aachen,4
1,Arsenal,12
2,Aston Villa,4
3,Augsburg,6
4,Bayern Munich,14


In [39]:
# Overwrote df_wins_away with a groupby of AwayTeam and count of each win.

df_wins_away = pd.DataFrame(df_wins_away.groupby('AwayTeam').FTR.count().reset_index())
df_wins_away.head()

,AwayTeam,FTR
0,Aachen,2
1,Arsenal,9
2,Aston Villa,3
3,Augsburg,2
4,Bayern Munich,9


In [40]:
# Merged or joined DataFrames df_wins_home and df_wins_away as a new DataFrame df_wins.
# change left to outer
df_wins = df_wins_home.merge(df_wins_away, how = 'outer', left_on = 'HomeTeam', right_on = 'AwayTeam')
df_wins.head()

,HomeTeam,FTR_x,AwayTeam,FTR_y
0,Aachen,4,Aachen,2
1,Arsenal,12,Arsenal,9
2,Aston Villa,4,Aston Villa,3
3,Augsburg,6,Augsburg,2
4,Bayern Munich,14,Bayern Munich,9


In [41]:
# Created a new column in df_wins titled Wins that adds the wins from FTR_x and FTR_y.

df_wins['Wins'] = df_wins.FTR_x + df_wins.FTR_y
df_wins.head()

,HomeTeam,FTR_x,AwayTeam,FTR_y,Wins
0,Aachen,4,Aachen,2,6
1,Arsenal,12,Arsenal,9,21
2,Aston Villa,4,Aston Villa,3,7
3,Augsburg,6,Augsburg,2,8
4,Bayern Munich,14,Bayern Munich,9,23


In [42]:
# Dropped three columns in df_wins FTR_x, AwayTeam, and FTR_y.

df_wins = df_wins.drop(columns = ['FTR_x', 'AwayTeam', 'FTR_y'])
df_wins.head()

,HomeTeam,Wins
0,Aachen,6
1,Arsenal,21
2,Aston Villa,7
3,Augsburg,8
4,Bayern Munich,23


In [43]:
# Renamed the column in df_wins HomeTeam to Teams.

df_wins = df_wins.rename(columns = {'HomeTeam': 'Teams'})
df_wins.head()

,Teams,Wins
0,Aachen,6
1,Arsenal,21
2,Aston Villa,7
3,Augsburg,8
4,Bayern Munich,23


In [44]:
# Merged or joined DataFrames df_goals and df_wins as a new DataFrame df_goals.

df_goals = df_goals.merge(df_wins, how = 'outer')
df_goals.head()
df_goals.nunique() ## I have 

Teams    56
Goals    37
Wins     21
dtype: int64

# Number of Loses


In [45]:
# DataFrame df_loses_home filters for home loss.

df_loses_home = df_matches[df_matches['FTR'] == 'A']
df_loses_home.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
7,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A
8,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A


In [46]:
# DataFrame df_loses_away filters for away loss.

df_loses_away = df_matches[df_matches['FTR'] == 'H']
df_loses_away.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
5,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H
9,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H
11,1103,D1,2011,2012-03-03,Leverkusen,Bayern Munich,2,0,H
16,1108,D1,2011,2012-04-11,Dortmund,Bayern Munich,1,0,H


In [59]:
# Overwrote df_loses_away with a groupby of HomeTeam and count of each win.

df_loses_home = pd.DataFrame(df_loses_home.groupby('HomeTeam').FTR.count().reset_index())
df_loses_home.head()

,HomeTeam,FTR
0,Aachen,1
1,Arsenal,1
2,Aston Villa,1
3,Augsburg,1
4,Bayern Munich,1


In [60]:
# Overwrote df_loses_away with a groupby of AwayTeam and count of each win.

df_loses_away = pd.DataFrame(df_loses_away.groupby('AwayTeam').FTR.count().reset_index())
df_loses_away.head()

,AwayTeam,FTR
0,Aachen,1
1,Arsenal,1
2,Aston Villa,1
3,Augsburg,1
4,Bayern Munich,1


In [49]:
# Merged or joined DataFrames df_loses_home and df_loses_away as a new DataFrame df_loses.

df_loses = df_loses_home.merge(df_loses_away, how = 'outer', left_on = 'HomeTeam', right_on = 'AwayTeam')
df_loses


,HomeTeam,FTR_x,AwayTeam,FTR_y
0,Aachen,7.0,Aachen,8
1,Arsenal,3.0,Arsenal,7
2,Aston Villa,8.0,Aston Villa,6
3,Augsburg,4.0,Augsburg,8
4,Bayern Munich,2.0,Bayern Munich,5
5,Blackburn,12.0,Blackburn,11
6,Bochum,7.0,Bochum,10
7,Bolton,11.0,Bolton,11
8,Braunschweig,3.0,Braunschweig,6
9,Chelsea,4.0,Chelsea,6


In [50]:
df_loses = df_loses.fillna(0)
df_loses

,HomeTeam,FTR_x,AwayTeam,FTR_y
0,Aachen,7.0,Aachen,8
1,Arsenal,3.0,Arsenal,7
2,Aston Villa,8.0,Aston Villa,6
3,Augsburg,4.0,Augsburg,8
4,Bayern Munich,2.0,Bayern Munich,5
5,Blackburn,12.0,Blackburn,11
6,Bochum,7.0,Bochum,10
7,Bolton,11.0,Bolton,11
8,Braunschweig,3.0,Braunschweig,6
9,Chelsea,4.0,Chelsea,6


In [51]:
# Created a new column in df_loses titled Wins that adds the wins from FTR_x and FTR_y.

df_loses['Loses'] = df_loses.FTR_x + df_loses.FTR_y
df_loses

,HomeTeam,FTR_x,AwayTeam,FTR_y,Loses
0,Aachen,7.0,Aachen,8,15.0
1,Arsenal,3.0,Arsenal,7,10.0
2,Aston Villa,8.0,Aston Villa,6,14.0
3,Augsburg,4.0,Augsburg,8,12.0
4,Bayern Munich,2.0,Bayern Munich,5,7.0
5,Blackburn,12.0,Blackburn,11,23.0
6,Bochum,7.0,Bochum,10,17.0
7,Bolton,11.0,Bolton,11,22.0
8,Braunschweig,3.0,Braunschweig,6,9.0
9,Chelsea,4.0,Chelsea,6,10.0


In [52]:
# Dropped three columns in df_loses FTR_x, AwayTeam, and FTR_y.

df_loses = df_loses.drop(columns = ['FTR_x', 'AwayTeam', 'FTR_y'])
df_loses

,HomeTeam,Loses
0,Aachen,15.0
1,Arsenal,10.0
2,Aston Villa,14.0
3,Augsburg,12.0
4,Bayern Munich,7.0
5,Blackburn,23.0
6,Bochum,17.0
7,Bolton,22.0
8,Braunschweig,9.0
9,Chelsea,10.0


In [53]:
# Renamed the column in df_loses HomeTeam to Teams.

df_loses = df_loses.rename(columns = {'HomeTeam': 'Teams'})
df_loses.head()
df_loses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Data columns (total 2 columns):
Teams    56 non-null object
Loses    56 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.3+ KB


In [54]:
# Merged or joined DataFrames df_goals and df_loses as a new DataFrame df_goals.

df_goals = df_goals.merge(df_loses , how = 'letf')
df_goals


KeyError: 'letf'

In [55]:
df_loses.head()

,Teams,Loses
0,Aachen,15.0
1,Arsenal,10.0
2,Aston Villa,14.0
3,Augsburg,12.0
4,Bayern Munich,7.0


Match_ID    992
Div           3
Season        1
Date        165
HomeTeam     56
AwayTeam     56
FTHG          9
FTAG          7
FTR           3
dtype: int64

In [57]:
 # Hannover" doest have any looses
# display(df_matches[df_matches['HomeTeam'] == 'Hannover'])
# display(df_matches[df_matches['AwayTeam'] == 'Hannover'])

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
9,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H
26,1118,D1,2011,2011-09-18,Hannover,Dortmund,2,1,H
101,1193,D1,2011,2012-03-18,Hannover,FC Koln,4,1,H
106,1198,D1,2011,2012-04-22,Hannover,Freiburg,0,0,D
130,1222,D1,2011,2011-11-26,Hannover,Hamburg,1,1,D
137,1229,D1,2011,2012-05-05,Hannover,Kaiserslautern,2,1,H
183,1275,D1,2011,2011-12-10,Hannover,Leverkusen,0,0,D
188,1280,D1,2011,2012-01-27,Hannover,Nurnberg,1,0,H
208,1300,D1,2011,2011-11-06,Hannover,Schalke 04,2,2,D
234,1326,D1,2011,2012-02-19,Hannover,Stuttgart,4,2,H


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
272,1364,D1,2011,2012-03-24,Bayern Munich,Hannover,2,1,H
273,1365,D1,2011,2011-11-19,Wolfsburg,Hannover,4,1,H
274,1366,D1,2011,2012-04-14,Hamburg,Hannover,1,0,H
275,1367,D1,2011,2011-10-16,FC Koln,Hannover,2,0,H
276,1368,D1,2011,2011-09-24,Augsburg,Hannover,0,0,D
277,1369,D1,2011,2012-04-08,Schalke 04,Hannover,3,0,H
278,1370,D1,2011,2011-12-18,Kaiserslautern,Hannover,1,1,D
279,1371,D1,2011,2011-08-13,Nurnberg,Hannover,1,2,A
280,1372,D1,2011,2012-04-28,Leverkusen,Hannover,1,0,H
281,1373,D1,2011,2012-02-26,Dortmund,Hannover,3,1,H


In [58]:
# Hannover loose 10 away but hannover didnt loose at home. this is why e are goignt have a null value???
df_loses_home

,HomeTeam,FTR
0,Aachen,7
1,Arsenal,3
2,Aston Villa,8
3,Augsburg,4
4,Bayern Munich,2
5,Blackburn,12
6,Bochum,7
7,Bolton,11
8,Braunschweig,3
9,Chelsea,4


#Figurate


In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline
hist = plt.hist(d)
plt.show()
    

 - The team's win percentage on days where it was raining during games in the 2011 season.


## Web Scrapping

In [ ]:
import requests
page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

In [ ]:
page.status_code


In [ ]:
if page.status_code == 200:
    print('go')

In [ ]:
key = 'cd1046ee98eec3e0f87367a0fbfcb817'

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!

In [67]:
df_matches.columns

Index(['Match_ID', 'Div', 'Season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG',
       'FTAG', 'FTR'],
      dtype='object')

In [71]:
df_matches.Date.dtype

dtype('O')

In [73]:
df_matches.Date

0      2012-03-31
1      2011-12-11
2      2011-08-13
3      2011-11-27
4      2012-02-18
5      2012-01-20
6      2012-02-04
7      2012-04-21
8      2011-09-18
9      2011-10-23
10     2011-10-01
11     2012-03-03
12     2011-08-27
13     2012-03-17
14     2011-11-06
15     2012-05-05
16     2012-04-11
17     2011-08-27
18     2011-12-17
19     2012-02-18
20     2012-02-03
21     2011-08-13
22     2011-10-29
23     2012-01-22
24     2011-12-03
25     2012-04-14
26     2011-09-18
27     2012-03-25
28     2012-03-10
29     2012-04-07
          ...    
962    2012-04-28
963    2012-04-28
964    2012-04-28
965    2012-04-29
966    2012-04-29
967    2012-04-30
968    2012-05-01
969    2012-05-01
970    2012-05-02
971    2012-05-02
972    2012-05-05
973    2012-05-06
974    2012-05-06
975    2012-05-06
976    2012-05-06
977    2012-05-06
978    2012-05-06
979    2012-05-06
980    2012-05-07
981    2012-05-08
982    2012-05-13
983    2012-05-13
984    2012-05-13
985    2012-05-13
986    201

In [79]:
import requests
import json 


In [80]:
resp = requests.get('https://darksky.net/dev')

In [81]:
resp


<Response [200]>

In [84]:
resp.text

'<!DOCTYPE html>\n<html>\n  <head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width,minimum-scale=1,initial-scale=1">\n    <title>Dark Sky \n</title>\n\n    <link href="/favicon.ico" rel="shortcut icon" sizes="32x32 16x16" type="image/x-icon">\n    <link href="https://fonts.googleapis.com/css?family=Lato:300,400,700,900" rel="stylesheet" type="text/css"/>\n    <link href="/dev/css/vendor/normalize.css" rel="stylesheet" type="text/css">\n    <link href="/dev/css/vendor/flexboxgrid.min.css" rel="stylesheet" type="text/css">\n    <link href="/dev/css/style.css" rel="stylesheet" type="text/css">\n    <link href="/dev/css/responsive.css" rel="stylesheet" type="text/css">\n\n    <script type="application/ld+json">\n      {\n        "@context": "http://schema.org",\n        "@type": "WebSite",\n        "name": "Dark Sky",\n        "url": "https://darksky.net"\n      }\n    </script>\n    <script type="application/ld+json">\n      {\n        "@context": "http:

In [85]:
data = json.loads(resp.text)
print(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)